In [7]:
from groq import Groq
import pandas as pd
import re
import os

In [83]:
df = pd.read_csv('../../data_acquisition/data/df_with_salary.csv')
df_with_salary_few_shot = pd.read_csv('../../data_acquisition/data/df_with_salary_few_shot.csv')

In [84]:
df.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000


Block 0: Few shot 5 examples with salary 
Block 1: Job title 
Block 2: company 
Block 3: workload (min workload and max workload) 
Block 4: Contract type 
Block 5: description

Notes one shot: 

df_with_salary_few_shot_12 = df_with_salary_few_shot[['Title', 'Company', 'Max_Salary', 'Min_Salary']]
    
    example_messages = [generate_message(row['Title'], row['Company'], row['Min_Salary'], row['Max_Salary']) for index, row in df_with_salary_few_shot_12.iterrows()]


# Set up for Usage

In [106]:
client = Groq(
    api_key="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",
)


def extract_salaries(response):
    # General regex pattern - covers most cases
    general_pattern = re.compile(r"(\d{1,3}(?:['\s]\d{3})*)\s*(?:-|und)\s*(\d{1,3}(?:['\s]\d{3})*)\s*CHF")
    # Special regex pattern - handles "CHF" before the numbers and potential markdown
    special_pattern = re.compile(r"CHF\s*(\d{1,3}(?:[',\s]\d{3})*)\s*[-–]\s*CHF\s*(\d{1,3}(?:[',\s]\d{3})*)")
    # New pattern for numbers inside parentheses with "und" and possibly no formatting
    parentheses_pattern = re.compile(r"\((\d{1,3}(?:['\s]?\d{3})*)\)\s*und\s*\((\d{1,3}(?:['\s]?\d{3})*)\)\s*CHF")
    # New pattern for a range inside parentheses with a dash
    range_parentheses_pattern = re.compile(r"\((\d{1,3}(?:['\s]\d{3})*)\s*-\s*(\d{1,3}(?:['\s]\d{3})*)\)\s*CHF")

    # Try the general pattern first
    matches = general_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary
    
    # If no matches, try the special pattern
    matches = special_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary

    # Try the new parentheses pattern
    matches = parentheses_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary

    # Try the range inside parentheses pattern
    matches = range_parentheses_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary

    return None, None

## LLM request with Block 1 and fewshot with Block 1 and salary

In [116]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_fewshot.csv'
min = 'block1_min_salary_fewshot'
max = 'block1_max_salary_fewshot'
answer = 'block1_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an. Der angegebene Gehaltsbereich sollte dabei eine maximale Spanne von 20000 CHF nicht überschreiten."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  liegt zwischen {min_salary} und {max_salary}."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
90000 110000
1
80000 100000
2
70000 90000
3
60000 80000
4
90000 110000
5
110000 130000
6
110000 130000
7
90000 110000
8
90000 110000
9
90000 110000
10
90000 110000
11
90000 110000
12
90000 110000
13
90000 110000
14
90000 110000
15
110000 130000
16
90000 110000
17
90000 110000
18
80000 100000
19
90000 110000
20
90000 110000
21
90000 110000
22
90000 110000
23
90000 110000
24
90000 110000
25
80000 100000
26
90000 110000
27
90000 110000
28
90000 110000
29
90000 110000
30
70000 90000
31
90000 110000
32
90000 110000
33
90000 110000
34
90000 110000
35
90000 110000
36
90000 110000
37
90000 110000
38
90000 110000
39
90000 110000
40
90000 110000
41
90000 110000
42
90000 110000
43
90000 110000
44
90000 110000
45
90000 110000
46
90000 110000
47
90000 110000
48
110000 130000
49
90000 110000
50
90000 110000
51
90000 110000
52
80000 100000
53
90000 110000
54
90000 110000
55
90000 110000
56
90000 110000
57
90000 110000
58
90000 110000
59
90000 110000
60
90000 110000
61
80000 100000
62
90000 110000
6

In [117]:
llm_block1_fewshot = pd.read_csv('data/llm_block1_fewshot.csv')
llm_block1_fewshot.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary_fewshot,block1_max_salary_fewshot,block1_answer_fewshot,block1_2_min_salary_fewshot,block1_2_max_salary_fewshot,block1_2_answer_fewshot
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,90000.0,110000.0,(90'000) und (110'000) CHF,90000,120000,90'000 und 120'000 CHF
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,80000.0,100000.0,(80'000) und (100'000) CHF,80000,110000,80'000 und 110'000 CHF
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,70000.0,90000.0,(70'000) und (90'000) CHF,60000,80000,60'000 und 80'000 CHF
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,60000.0,80000.0,Schätzung: 60'000 - 80'000 CHF,45000,60000,45'000 und 60'000 CHF
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,90000.0,110000.0,90'000 - 110'000 CHF,90000,110000,(90'000) und (110'000) CHF


In [ ]:
llm_block1_fewshot[llm_block1_fewshot['block1_min_salary_fewshot'].isnull() & llm_block1_fewshot['block1_max_salary_fewshot'].isnull()]

## LLM request with Block 1,2 and fewshot with Block 1,2 and salary

In [119]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_fewshot.csv'
min = 'block1_2_min_salary_fewshot'
max = 'block1_2_max_salary_fewshot'
answer = 'block1_2_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an. Der angegebene Gehaltsbereich sollte dabei eine maximale Spanne von 20000 CHF nicht überschreiten."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  bei der Firma {company} liegt zwischen {min_salary} und {max_salary}."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
90000 110000
1
80000 100000
2
60000 80000
3
60000 80000
4
90000 110000
5
110000 130000
6
90000 110000
7
90000 110000
8
90000 110000
9
90000 110000
10
110000 130000
11
90000 110000
12
90000 110000
13
90000 110000
14
90000 110000
15
110000 130000
16
90000 110000
17
90000 110000
18
70000 90000
19
90000 110000
20
90000 110000
21
90000 110000
22
90000 110000
23
90000 110000
24
90000 110000
25
80000 100000
26
80000 100000
27
90000 110000
28
90000 110000
29
80000 100000
30
75000 95000
31
90000 110000
32
90000 110000
33
90000 110000
34
80000 100000
35
90000 110000
36
90000 110000
37
90000 110000
38
90000 110000
39
80000 100000
40
80000 100000
41
90000 110000
42
90000 110000
43
80000 100000
44
90000 110000
45
90000 110000
46
90000 110000
47
90000 110000
48
110000 130000
49
90000 110000
50
90000 110000
51
90000 110000
52
70000 90000
53
90000 110000
54
90000 110000
55
90000 110000
56
90000 110000
57
90000 110000
58
90000 110000
59
90000 110000
60
90000 110000
61
80000 100000
62
90000 110000
63


In [120]:
llm_block1_2_fewshot = pd.read_csv('data/llm_block1_2_fewshot.csv')
llm_block1_2_fewshot.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary_fewshot,block1_max_salary_fewshot,block1_answer_fewshot,block1_2_min_salary_fewshot,block1_2_max_salary_fewshot,block1_2_answer_fewshot
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,90000.0,110000.0,(90'000) und (110'000) CHF,90000.0,110000.0,(90'000) und (110'000) CHF
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,80000.0,100000.0,(80'000) und (100'000) CHF,80000.0,100000.0,80'000 und 100'000 CHF
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,70000.0,90000.0,(70'000) und (90'000) CHF,60000.0,80000.0,(60'000) und (80'000) CHF
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,60000.0,80000.0,Schätzung: 60'000 - 80'000 CHF,60000.0,80000.0,(60'000 - 80'000) CHF
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,90000.0,110000.0,90'000 - 110'000 CHF,90000.0,110000.0,(90'000) und (110'000) CHF


In [121]:
llm_block1_2_fewshot[llm_block1_2_fewshot['block1_2_min_salary_fewshot'].isnull() & llm_block1_2_fewshot['block1_2_min_salary_fewshot'].isnull()]

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary_fewshot,block1_max_salary_fewshot,block1_answer_fewshot,block1_2_min_salary_fewshot,block1_2_max_salary_fewshot,block1_2_answer_fewshot
101,"Director (all genders), SAP Supply Chain Consu...",JOB SCHWEIZ AG,Festanstellung,"Introduction We are JOB SCHWEIZ AG, a recruit...",80,100,200000,200000,NaN,NaN,120000 und 140000 CHF,120000,150000,(120000) und (150000) CHF


## LLM request with Block 1,2,3 and fewshot with Block 1,2,3 and salary


In [122]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_3_fewshot.csv'
min = 'block1_2_3_min_salary_fewshot'
max = 'block1_2_3_max_salary_fewshot'
answer = 'block1_2_3_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Mit einer Arbeitszeit von {row['min_workload']} bis {row['max_workload']} Prozent. Wenn du das genaue Gehalt nicht kennst, gib bitte eine Schätzung an. Der angegebene Gehaltsbereich sollte dabei eine maximale Spanne von 20000 CHF nicht überschreiten."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  bei der Firma {company} mit einer Arbeitszeit von {min_workload} bis {max_workload} Prozent liegt zwischen {min_salary} und {max_salary}. "
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
90000 110000
1
70000 90000
2
70000 90000
3
None None
4
90000 110000
5
110000 130000
6
90000 110000
7
90000 110000
8
90000 110000
9
90000 110000
10
110000 130000
11
90000 110000
12
90000 110000
13
90000 110000
14
90000 110000
15
110000 130000
16
90000 110000
17
90000 110000
18
70000 90000
19
90000 110000
20
90000 110000
21
90000 110000
22
90000 110000
23
90000 110000
24
80000 100000
25
80000 100000
26
80000 100000
27
90000 110000
28
90000 110000
29
80000 100000
30
75000 95000
31
90000 110000
32
90000 110000
33
90000 110000
34
80000 100000
35
90000 110000
36
90000 110000
37
90000 110000
38
90000 110000
39
90000 110000
40
90000 110000
41
90000 110000
42
90000 110000
43
90000 110000
44
90000 110000
45
90000 110000
46
90000 110000
47
90000 110000
48
90000 110000
49
90000 110000
50
90000 110000
51
90000 110000
52
80000 100000
53
90000 110000
54
90000 110000
55
90000 110000
56
90000 110000
57
90000 110000
58
90000 110000
59
90000 110000
60
90000 110000
61
80000 100000
62
90000 110000
63
900

In [123]:
llm_block1_2_3_fewshot = pd.read_csv('data/llm_block1_2_3_fewshot.csv')
llm_block1_2_3_fewshot.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary_fewshot,block1_max_salary_fewshot,block1_answer_fewshot,block1_2_min_salary_fewshot,block1_2_max_salary_fewshot,block1_2_answer_fewshot,block1_2_3_min_salary_fewshot,block1_2_3_max_salary_fewshot,block1_2_3_answer_fewshot
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,90000.0,110000.0,(90'000) und (110'000) CHF,90000.0,110000.0,(90'000) und (110'000) CHF,90000.0,110000.0,(90'000) und (110'000) CHF
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,80000.0,100000.0,(80'000) und (100'000) CHF,80000.0,100000.0,80'000 und 100'000 CHF,70000.0,90000.0,(70000) und (90000) CHF
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,70000.0,90000.0,(70'000) und (90'000) CHF,60000.0,80000.0,(60'000) und (80'000) CHF,70000.0,90000.0,(70000) und (90000) CHF
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,60000.0,80000.0,Schätzung: 60'000 - 80'000 CHF,60000.0,80000.0,(60'000 - 80'000) CHF,NaN,NaN,45000 und 65000 CHF
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,90000.0,110000.0,90'000 - 110'000 CHF,90000.0,110000.0,(90'000) und (110'000) CHF,90000.0,110000.0,(90'000) und (110'000) CHF


In [124]:
llm_block1_2_3_fewshot[llm_block1_2_3_fewshot['block1_2_3_min_salary_fewshot'].isnull() & llm_block1_2_3_fewshot['block1_2_3_min_salary_fewshot'].isnull()]

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_min_salary_fewshot,block1_max_salary_fewshot,block1_answer_fewshot,block1_2_min_salary_fewshot,block1_2_max_salary_fewshot,block1_2_answer_fewshot,block1_2_3_min_salary_fewshot,block1_2_3_max_salary_fewshot,block1_2_3_answer_fewshot
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,60000.0,80000.0,Schätzung: 60'000 - 80'000 CHF,60000.0,80000.0,(60'000 - 80'000) CHF,NaN,NaN,45000 und 65000 CHF


## LLM request with Block 1,2,3,4 and fewshot with Block 1,2,3,4 and salary


In [ ]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_3_4_fewshot.csv'
min = 'block1_2_3_4_min_salary_fewshot'
max = 'block1_2_3_4_max_salary_fewshot'
answer = 'block1_2_3_4_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Mit einer Arbeitszeit von {row['min_workload']} bis {row['max_workload']} Prozent  und der Vertragsart {row['contract_type']}. Wenn du das genaue Gehalt nicht kennst, gib bitte eine Schätzung an. Der angegebene Gehaltsbereich sollte dabei eine maximale Spanne von 20000 CHF nicht überschreiten."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, contract_type,description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  bei der Firma {company} mit einer Arbeitszeit von {min_workload} bis {max_workload} Prozent und der Vertragsart {contract_type} liegt zwischen {min_salary} und {max_salary}. "
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description'], row['contract_type']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
90000 110000
1
80000 100000
2
80000 100000
3
None None
4
90000 110000
5
120000 140000
6
120000 140000
7
110000 130000
8
90000 110000
9
90000 110000
10
110000 130000
11
120000 140000
12
110000 130000
13
90000 110000
14
120000 140000
15
120000 140000


In [ ]:
llm_block1_2_3_4_fewshot = pd.read_csv('data/llm_block1_2_3_4_fewshot.csv')
llm_block1_2_3_4_fewshot.head()

In [ ]:
llm_block1_2_3_4_fewshot[llm_block1_2_3_fewshot['block1_2_3_4_min_salary_fewshot'].isnull() & llm_block1_2_3_4_fewshot['block1_2_3_4_min_salary_fewshot'].isnull()]

## LLM request with Block 1,2,3,4,5 and fewshot with Block 1,2,3,4,5 and salary


In [ ]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_3_4_5_fewshot.csv'
min = 'block1_2_3_4_5_min_salary_fewshot'
max = 'block1_2_3_4_5_max_salary_fewshot'
answer = 'block1_2_3_4_5_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Mit einer Arbeitszeit von {row['min_workload']} bis {row['max_workload']} Prozent  und der Vertragsart {row['contract_type']}. Hier ist die Beschreibung, was Sie benötigen und tun müssen: {row['description']}. Wenn du das genaue Gehalt nicht kennst, gib bitte eine Schätzung an. Der angegebene Gehaltsbereich sollte dabei eine maximale Spanne von 20000 CHF nicht überschreiten."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  bei der Firma {company} mit einer Arbeitszeit von {min_workload} bis {max_workload} Prozent und der Vertragsart {contract_type} liegt zwischen {min_salary} und {max_salary}. Hier ist die Beschreibung, was Sie benötigen und tun müssen: {description} "
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description'], row['contract_type']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

In [ ]:
llm_block1_2_3_4_5_fewshot = pd.read_csv('data/llm_block1_2_3_4_5_fewshot.csv')
llm_block1_2_3_4_5_fewshot.head()

In [ ]:
llm_block1_2_3_4_5_fewshot[llm_block1_2_3_fewshot['block1_2_3_4_5_min_salary_fewshot'].isnull() & llm_block1_2_3_4_5_fewshot['block1_2_3_4_5_min_salary_fewshot'].isnull()]